# Libraries

In [1]:
import os
from pathlib import Path
import numpy as np
import pickle
import pandas as pd

from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
import scipy

In [2]:
features_path = Path.cwd()
notebooks_path = features_path.parent
repo_path = notebooks_path.parent
os.chdir(str(features_path))
#print current working directory
print(notebooks_path)

/home/ricardino/Documents/MAIA/tercer_semestre/CAD/Projecte/Machine_Learning/notebooks


# Functions and classes

In [3]:
class path_label():
    """Class to access paths and labels from csv
    """
    def __init__(self, meta=pd.read_csv(str(repo_path) + '/data/meta_info.csv', sep='\t'), classif='binary', set_name='train') -> None:
        meta = meta.loc[meta['classif'] == classif] #Filter by classif
        meta = meta.loc[meta['set'] == set_name] #Filter by set
        self.paths = list(meta.path)
        self. labels = np.array(meta.label)
        self.FOV_x1 = np.array(meta.FOV_x1, dtype=np.int16)
        self.FOV_x2 = np.array(meta.FOV_x2, dtype=np.int16)
        self.FOV_y1 = np.array(meta.FOV_y1, dtype=np.int16)
        self.FOV_y2 = np.array(meta.FOV_y2, dtype=np.int16)

In [4]:
def save_pickle(file, filename):
    """save as pickle

    Args:
        file (obj): object to save
        filename (str): path of the object to save
    """
    with open(filename, 'wb') as handle:
        pickle.dump(file, handle, pickle.HIGHEST_PROTOCOL)

def open_pickle(filename):
    """open pickle file

    Args:
        filename (str): path of pickle file

    Returns:
        obj: object extracted form pickle
    """
    with open(filename, 'rb') as handle:
        return pickle.load(handle)

In [5]:
def classifier(method, grid_type='normal', scaler=StandardScaler(), reductor_components = [0.95,0.99], verbose=4):
    """get pipe and grid for classifier

    Args:
        method (str): classifier name
        scaler (scikit object, optional): type of scaler. Defaults to StandardScaler().
        verbose (int, optional): verbose level. Defaults to 4.

    Returns:
        _type_: _description_
    """
   #KNN
    if(method=='KNN'):
        param_grid = {'classifier__n_neighbors': list(range(1,40))}
        pipe = Pipeline([('scaler', scaler),('classifier',KNeighborsClassifier())])
        grid = GridSearchCV(pipe, param_grid, verbose = verbose)
        return grid, pipe
    if(method=='KNN_PCA'):
        param_grid = {'reductor__n_components': reductor_components, 'classifier__n_neighbors': list(range(1,40))}
        pipe = Pipeline([('scaler', scaler),('reductor',PCA()), ('classifier',KNeighborsClassifier())])
        grid = GridSearchCV(pipe, param_grid, verbose = verbose)
        return grid, pipe
    #RF
    elif(method=='RF'):
        param_grid = {'classifier__n_estimators': [100, 200, 400],}
        pipe = Pipeline([('scaler', scaler),('classifier',RandomForestClassifier())])
        grid = GridSearchCV(pipe, param_grid, verbose = verbose)
        return grid, pipe
    
    elif(method=='RF_PCA'):
        param_grid = {'reductor__n_components': reductor_components,'classifier__n_estimators': [100, 200, 400],}
        pipe = Pipeline([('scaler', scaler),('reductor',PCA()), ('classifier',RandomForestClassifier())])
        grid = GridSearchCV(pipe, param_grid, scoring='accuracy', verbose = verbose)
        return grid, pipe
    #SVM
    elif(method=='SVM'):
        if(grid_type=='normal'):
            param_grid = {'classifier__C': [0.1, 1, 10, 100],
                    'classifier__gamma': [100, 10, 1, 0.1, 0.01, 0.001, 0.0001],
                    'classifier__kernel': ['rbf'],'classifier__class_weight':['balanced']}
            pipe = Pipeline([('scaler', scaler),('classifier',SVC())])
            grid = GridSearchCV(pipe, param_grid, scoring='accuracy', verbose = verbose)
            return grid, pipe
        elif(grid_type=='random'):
            parameters = {'classifier__C': scipy.stats.expon(scale=10), 'classifier__gamma': scipy.stats.expon(scale=.001), #Parameters for grid search
            'classifier__kernel': ['rbf'], 'classifier__class_weight':['balanced']}
            pipe = Pipeline([('scaler', scaler),('classifier',SVC())]) #Definition of pipeline
            grid = RandomizedSearchCV(pipe, parameters,n_iter=20, scoring='accuracy', verbose=verbose, return_train_score=False) #Random search
            return grid, pipe
    elif(method=='SVM_PCA'):
        if(grid_type=='normal'):
            param_grid = {'reductor__n_components': reductor_components, 'classifier__C': [0.1, 1, 10, 100],
                    'classifier__gamma': [100, 10, 1, 0.1, 0.01, 0.001, 0.0001],
                    'classifier__kernel': ['rbf'],'classifier__class_weight':['balanced']}
            pipe = Pipeline([('scaler', scaler),('reductor',PCA()), ('classifier',SVC())])
            grid = GridSearchCV(pipe, param_grid, scoring='accuracy', verbose = verbose)
            return grid, pipe
        elif(grid_type=='random'):
            parameters = {'reductor__n_components': reductor_components, 'classifier__C': scipy.stats.expon(scale=10), 'classifier__gamma': scipy.stats.expon(scale=.001), #Parameters for grid search
            'classifier__kernel': ['rbf'], 'classifier__class_weight':['balanced']}
            pipe = Pipeline([('scaler', scaler),('reductor',PCA()), ('classifier',SVC())]) #Definition of pipeline
            grid = RandomizedSearchCV(pipe, parameters,n_iter=100, scoring='accuracy', verbose=verbose, return_train_score=False) #Random search
            return grid, pipe
    #ADA
    elif(method=='AdaBoost'):
        param_grid = {'classifier__n_estimators': [50,100, 200, 400], 'classifier__learning_rate': [ 0.1, 1,5, 10]}
        pipe = Pipeline([('scaler', scaler),('classifier',AdaBoostClassifier())])
        grid = GridSearchCV(pipe, param_grid, verbose = verbose)
        return grid, pipe
    elif(method=='AdaBoost_PCA'):
        param_grid = {'reductor__n_components': reductor_components, 'classifier__n_estimators': [50,100, 200, 400], 'classifier__learning_rate': [ 0.1, 1,5, 10]}
        pipe = Pipeline([('scaler', scaler),('reductor',PCA()),('classifier',AdaBoostClassifier())])
        grid = GridSearchCV(pipe, param_grid, verbose = verbose)
        return grid, pipe
    #GradientBoosting
    elif(method=='GradBoost'):
        param_grid = {'classifier__learning_rate': [0.01, 0.1, 1, 10], 'classifier__max_iter': [100, 200, 400]}
        pipe = Pipeline([('scaler', scaler), ('classifier',GradientBoostingClassifier())])
        grid = GridSearchCV(pipe, param_grid, verbose = verbose)
        return grid, pipe


# IMP

In [6]:
def color_data_load(classif, color_space, f_name):
    # Read train and valiadation data
    meta = pd.read_csv(str(repo_path) + '/data/meta_info.csv', sep='\t') #For labels
    f_type = 'color'

    #We read the training data
    X_train = open_pickle(str(repo_path)+ f'/data/features/{f_type}/{color_space}/{classif}_train_{f_type}_{f_name}_fv.p')
    y_train = path_label(meta, classif, set_name='train').labels
    #Print shapes to be sure that dimensions are the same
    print(f'The shape of the training data is {X_train.shape} and its labels are {y_train.shape}')

    #We read the validation data
    X_val = open_pickle(str(repo_path)+ f'/data/features/{f_type}/{color_space}/{classif}_val_{f_type}_{f_name}_fv.p')
    y_val = path_label(meta, classif, set_name='val').labels
    print(f'The shape of the valdiation data is {X_val.shape} and its labels are {y_val.shape}')
    
    return X_train, X_val, y_train, y_val

In [7]:
def texture_data_load(classif, f_name):
    # Read train and valiadation data
    meta = pd.read_csv(str(repo_path) + '/data/meta_info.csv', sep='\t') #For labels
    f_type = 'texture'
    X_train = open_pickle(str(repo_path) + f'/data/features/{f_type}/{classif}_train_{f_type}_{f_name}_fv.p')
    y_train = path_label(meta, classif, set_name='train').labels
    if isinstance(X_train, list):
        X_train = np.array(X_train)
    #Print shapes to be sure that dimensions are the same
    print(f'The shape of the training data is {X_train.shape} and its labels are {y_train.shape}')
    
    #We read the validation data
    X_val = open_pickle(str(repo_path) + f'/data/features/{f_type}/{classif}_val_{f_type}_{f_name}_fv.p')
    y_val = path_label(meta, classif, set_name='val').labels
    if isinstance(X_val, list):
        X_val = np.array(X_val)
    print(f'The shape of the valdiation data is {X_val.shape} and its labels are {y_val.shape}')
    
    return X_train, X_val, y_train, y_val

## Data load

In [18]:
def get_Train_Val_Data(classif, c_name, t_name):
    #Color data load
    color_space = 'RGB'; f_name = 'ColorStats' #To locate feature matrices
    c = color_data_load(classif, color_space, c_name) #Get data
    #Texture data load
    f_name = '27'
    if t_name==None:
        t = None
        return c
    else:
        t = texture_data_load(classif, t_name)

    #Stack together
    X_train = np.hstack((c[0], t[0]))
    X_val = np.hstack((c[1], t[1]))
    y_train = c[2]
    y_val = c[3]
    print(f'The shape of the training data is {X_train.shape} and its labels are {y_train.shape}')
    print(f'The shape of the valdiation data is {X_val.shape} and its labels are {y_val.shape}')
    
    return X_train, X_val, y_train, y_val

In [26]:
classif = 'binary'
c_name = 'ColorStats'
t_name = '27'
X_train, X_val, y_train, y_val = get_Train_Val_Data(classif, c_name, t_name)

The shape of the training data is (15195, 54) and its labels are (15195,)
The shape of the valdiation data is (3796, 54) and its labels are (3796,)
The shape of the training data is (15195, 27) and its labels are (15195,)
The shape of the valdiation data is (3796, 27) and its labels are (3796,)
The shape of the training data is (15195, 81) and its labels are (15195,)
The shape of the valdiation data is (3796, 81) and its labels are (3796,)


## Training and testing

In [27]:
grid, pipe = classifier('RF', grid_type='random', scaler=StandardScaler(), verbose=4)
model = grid.fit(X_train,y_train)
save_pickle(model, str(repo_path) + f'/data/models/{classif}_{pipe.steps[-1][-1]}_{c_name}_{t_name}.p') #Save model
y_pred = model.predict(X_val) #predict
acc = np.mean(y_pred == y_val) #accuracy
print(acc)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ......classifier__n_estimators=100;, score=0.855 total time=   5.1s
[CV 2/5] END ......classifier__n_estimators=100;, score=0.832 total time=   5.2s
[CV 3/5] END ......classifier__n_estimators=100;, score=0.775 total time=   5.2s
[CV 4/5] END ......classifier__n_estimators=100;, score=0.735 total time=   5.3s
[CV 5/5] END ......classifier__n_estimators=100;, score=0.766 total time=   5.4s
[CV 1/5] END ......classifier__n_estimators=200;, score=0.860 total time=  11.1s
[CV 2/5] END ......classifier__n_estimators=200;, score=0.833 total time=  11.2s
[CV 3/5] END ......classifier__n_estimators=200;, score=0.776 total time=  11.1s
[CV 4/5] END ......classifier__n_estimators=200;, score=0.735 total time=  11.2s
[CV 5/5] END ......classifier__n_estimators=200;, score=0.771 total time=  10.9s
[CV 1/5] END ......classifier__n_estimators=400;, score=0.862 total time=  21.5s
[CV 2/5] END ......classifier__n_estimators=400;,